In [141]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

In [142]:
meteostat_1 = pd.read_csv('processed/442300170322_daily_meteostat.csv')
meteostat_1

,READ_DATE,KWH_EXP,tavg,tmin,tmax,wdir,wspd,pres
0,2019-01-01,11.001903,25.7,24.8,30.2,44.0,6.4,1008.9
1,2019-01-02,11.422662,26.0,24.1,29.2,21.0,5.4,1009.2
2,2019-01-03,2.341671,24.6,24.0,27.3,340.0,4.1,1010.3
3,2019-01-04,4.800008,26.2,23.0,29.8,340.0,6.9,1011.4
4,2019-01-05,5.752651,26.9,23.6,30.2,284.0,6.3,1011.7
...,...,...,...,...,...,...,...,...
1973,2024-05-27,2.381903,28.3,24.4,34.0,126.0,9.9,1009.9
1974,2024-05-28,2.412923,28.2,24.1,33.9,125.0,8.0,1009.4
1975,2024-05-29,2.564494,29.0,24.2,33.0,130.0,7.7,1009.1
1976,2024-05-30,2.558021,29.7,25.0,35.0,106.0,7.8,1009.7


In [143]:
df_weather = pd.read_csv('processed/weather_2019_2024_standard.csv')
df_weather

,Tanggal,Tavg,RH_avg,RR,ss
0,2019-01-01,25.7,96.0,12.9,7.5
1,2019-01-02,25.9,90.0,42.7,6.7
2,2019-01-03,24.9,94.0,0.5,0.1
3,2019-01-04,25.8,88.0,34.6,0.1
4,2019-01-05,26.7,88.0,2.1,3.2
...,...,...,...,...,...
2003,2024-06-26,27.4,76.0,0.0,8.0
2004,2024-06-27,27.8,77.0,0.0,9.9
2005,2024-06-28,28.4,76.0,0.0,10.8
2006,2024-06-29,28.6,65.0,0.0,10.0


In [144]:
meteostat_1['READ_DATE'] = pd.to_datetime(meteostat_1['READ_DATE'])
df_weather['Tanggal'] = pd.to_datetime(df_weather['Tanggal'])

meteostat_1['READ_DATE_day'] = meteostat_1['READ_DATE'].dt.date
df_weather['Tanggal'] = df_weather['Tanggal'].dt.date

meteostat_1 = pd.merge(meteostat_1, df_weather, how='left', left_on='READ_DATE_day', right_on='Tanggal')
meteostat_1.drop(columns=['Tanggal'], inplace=True)

meteostat_1.drop(columns=['READ_DATE_day'], inplace=True)

In [145]:
meteostat_1

,READ_DATE,KWH_EXP,tavg,tmin,tmax,wdir,wspd,pres,Tavg,RH_avg,RR,ss
0,2019-01-01,11.001903,25.7,24.8,30.2,44.0,6.4,1008.9,25.7,96.0,12.9,7.5
1,2019-01-02,11.422662,26.0,24.1,29.2,21.0,5.4,1009.2,25.9,90.0,42.7,6.7
2,2019-01-03,2.341671,24.6,24.0,27.3,340.0,4.1,1010.3,24.9,94.0,0.5,0.1
3,2019-01-04,4.800008,26.2,23.0,29.8,340.0,6.9,1011.4,25.8,88.0,34.6,0.1
4,2019-01-05,5.752651,26.9,23.6,30.2,284.0,6.3,1011.7,26.7,88.0,2.1,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...
1973,2024-05-27,2.381903,28.3,24.4,34.0,126.0,9.9,1009.9,28.7,73.0,0.0,10.4
1974,2024-05-28,2.412923,28.2,24.1,33.9,125.0,8.0,1009.4,28.2,75.0,0.0,8.2
1975,2024-05-29,2.564494,29.0,24.2,33.0,130.0,7.7,1009.1,29.0,77.0,0.0,10.7
1976,2024-05-30,2.558021,29.7,25.0,35.0,106.0,7.8,1009.7,29.6,73.0,0.0,10.6


In [146]:
result = seasonal_decompose(meteostat_1['KWH_EXP'], model='additive', period=7)

meteostat_1['trend'] = result.trend
meteostat_1['seasonal'] = result.seasonal
meteostat_1['residual'] = result.resid

In [147]:
meteostat_1.isna().sum()

READ_DATE    0
KWH_EXP      0
tavg         0
tmin         0
tmax         0
wdir         0
wspd         0
pres         0
Tavg         0
RH_avg       0
RR           0
ss           0
trend        6
seasonal     0
residual     6
dtype: int64

In [148]:
meteostat_1['trend'].fillna(meteostat_1['trend'].mean(), inplace=True)
meteostat_1['residual'].fillna(meteostat_1['residual'].mean(), inplace=True)
meteostat_1.fillna(0, inplace=True)

/tmp/ipykernel_4566/1491972629.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  meteostat_1['trend'].fillna(meteostat_1['trend'].mean(), inplace=True)
/tmp/ipykernel_4566/1491972629.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value

In [149]:
meteostat_1

,READ_DATE,KWH_EXP,tavg,tmin,tmax,wdir,wspd,pres,Tavg,RH_avg,RR,ss,trend,seasonal,residual
0,2019-01-01,11.001903,25.7,24.8,30.2,44.0,6.4,1008.9,25.7,96.0,12.9,7.5,27.581929,-0.127576,-0.000641
1,2019-01-02,11.422662,26.0,24.1,29.2,21.0,5.4,1009.2,25.9,90.0,42.7,6.7,27.581929,0.024909,-0.000641
2,2019-01-03,2.341671,24.6,24.0,27.3,340.0,4.1,1010.3,24.9,94.0,0.5,0.1,27.581929,-0.291080,-0.000641
3,2019-01-04,4.800008,26.2,23.0,29.8,340.0,6.9,1011.4,25.8,88.0,34.6,0.1,7.780154,0.521332,-3.501478
4,2019-01-05,5.752651,26.9,23.6,30.2,284.0,6.3,1011.7,26.7,88.0,2.1,3.2,7.810439,0.181104,-2.238892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973,2024-05-27,2.381903,28.3,24.4,34.0,126.0,9.9,1009.9,28.7,73.0,0.0,10.4,2.460534,-0.287815,0.209184
1974,2024-05-28,2.412923,28.2,24.1,33.9,125.0,8.0,1009.4,28.2,75.0,0.0,8.2,2.469894,-0.127576,0.070605
1975,2024-05-29,2.564494,29.0,24.2,33.0,130.0,7.7,1009.1,29.0,77.0,0.0,10.7,27.581929,0.024909,-0.000641
1976,2024-05-30,2.558021,29.7,25.0,35.0,106.0,7.8,1009.7,29.6,73.0,0.0,10.6,27.581929,-0.291080,-0.000641


In [150]:
# save to csv
meteostat_1.to_csv('processed/a_442300170322_engineered.csv', index=False)